In [1]:
## Name: Chandni Patel
## ID: A20455322
## CS 512 - Fall 2020
## Non-planar Camera Calibration

import numpy as np
import random
import math
import cv2
np.set_printoptions(formatter={'float': "{0:.4f}".format})

## Non-Planar Camera Calibration

In [2]:
#input point pairs
def GetFilePoint(filename):    
    point_3D, point_2D = [], []
    with open(filename) as f:
        point_pairs = f.readlines()
        for i in point_pairs:
            pt = i.split()
            point_3D.append([float(p) for p in pt[:3]])
            point_2D.append([float(p) for p in pt[3:]])
    print("\nTotal Point Pairs = ", len(point_3D))
    return point_3D, point_2D


In [3]:
#required matrix A
def GetMatrixA(point_3D, point_2D):
    A = []
    array_0 = np.zeros(4)
    for i, j in zip(point_3D, point_2D):        
        #convert 3D to 3DH
        pi = np.concatenate([np.array(i), [1]])
        #row 1 for point i
        xipi = j[0] * pi
        r1 = np.concatenate([pi, array_0, -xipi])
        A.append(r1)
        #row 2 for point i
        yipi = j[1] * pi
        r2 = np.concatenate([array_0, pi, -yipi])        
        A.append(r2)
    return A


In [4]:
#projection matrix M
def GetProjectionMatrixM(A):    
    M = []
    u, d, vT = np.linalg.svd(A, full_matrices = True)
    #get M from s i.e. vector with the singular values
    M = vT[-1].reshape(3, 4)   
    for i in range(len(d)):
        if (round(d[i],1) == 0):
            M = vT[i].reshape(3, 4)
            break
    return M


In [5]:
#mean square error
def GetMSE(point_3D, point_2D, M):    
    sum_error = 0
    m1, m2, m3 = M[0][:4], M[1][:4], M[2][:4] 
    for i, j in zip(point_3D, point_2D):
        #convert 3D to 3DH
        pi = np.concatenate([np.array(i), [1]])
        #compute xi & yi using M
        computed_xi = (m1.T.dot(pi)) / (m3.T.dot(pi))
        computed_yi = (m2.T.dot(pi)) / (m3.T.dot(pi))
        #sum of all errors
        sum_error += ((j[0] - computed_xi) ** 2 + (j[1] - computed_yi) ** 2)
    #E = (sum of errors) / m
    mse = sum_error / len(point_3D)
    print("\nMean Square Error = ", round(mse, 4))
    

In [6]:
def Non_Planar_Calibration(filename, WithRANSAC = False): 
    print("\n\n*************************************************************")
    point_3D, point_2D = GetFilePoint(filename)
    
    A, M = [], []    
    if WithRANSAC:
        print("\nCamera Parameters from Non-Planar Camera Calibration with RANSAC: ")
        A, M = ApplyRANSAC(point_3D, point_2D)        
    else:        
        print("\nCamera Parameters from Non-Planar Camera Calibration: ")
        A = GetMatrixA(point_3D, point_2D)
        M = GetProjectionMatrixM(A)        
    
    #additional variables
    a1 = M[0][:3] #vector a1
    a2 = M[1][:3] #vector a2
    a3 = M[2][:3] #vector a3
    b = [] #vector b
    for i in range(len(M)):
        b.append(M[i][3])       
    print("\nM^ = ", M) 

    #|p| = 1 / |a3|
    norm_rho = 1 / np.linalg.norm(a3)
    #u0 = |p|^2 a1.a3    
    u0 = norm_rho ** 2 * (a1.dot(a3.T))    
    #v0 = |p|^2 a2.a3
    v0 = norm_rho ** 2 * (a2.dot(a3.T))
    print("\nu0 = ", round(u0,4), "\t\tv0 = ", round(v0,4))
    
    #av = (|p|^2 a2.a2 - v0^2)^1/2
    av = np.sqrt(norm_rho ** 2 * a2.dot(a2.T) - v0 ** 2)    
    #s = (|p|^4 / av)[(a1 x a3).(a2 x a3)]
    s = (norm_rho ** 4) / av * np.cross(a1, a3).dot(np.cross(a2, a3))
    #au = (|p|^2 a1.a1 - u0^2 - s^2)^1/2
    au = np.sqrt(norm_rho ** 2 * a1.dot(a1.T) - u0 ** 2 - s ** 2)
    print("\nalpha u = ", round(au,4), "\talpha v = ", round(av,4))
    
    #sign of rho
    sign_rho = np.sign(b[2])
    rho = sign_rho * norm_rho
    print("\ns = ", round(s,4), "\t\tρ = ", round(rho,4))
    
    #get K*
    K_star = np.array([[au, s, u0],[0, av, v0],[0, 0, 1]])
    print("\nK* = ", K_star)    
    
    #T* = sign_rho |p| (K*)^-1 b
    T_star = rho * np.linalg.inv(K_star).dot(b).T
    print("\nT* = ", T_star)
    
    #r3 = sign_rho |p| a3
    r3 = rho * a3.T
    #r1 = (|p|^2 / av) (a2 x a3)
    r1 = norm_rho ** 2 / av * np.cross(a2, a3)
    #r2 = r3 x r1
    r2 = np.cross(r3, r1)
    #R* = [r1^T r2^T r3^T]^T
    R_star = np.array([r1.T, r2.T, r3.T])
    print("\nR* = ", R_star)
    
    #projection matrix M = pM^
    M = rho * M
    print("\nM = ", M)
    
    GetMSE(point_3D, point_2D, M)
    

In [7]:
#Camera Calibration without RANSAC
Non_Planar_Calibration('points_3Dto2D.txt', False)




*************************************************************

Total Point Pairs =  49

Camera Parameters from Non-Planar Camera Calibration: 

M^ =  [[0.0063 -0.0020 0.6415 0.6415]
 [0.0012 0.0054 0.2973 0.2973]
 [-0.0000 -0.0000 0.0038 0.0038]]

u0 =  167.3234 		v0 =  77.5282

alpha u =  1.7093 	alpha v =  1.482

s =  -0.0457 		ρ =  260.8117

K* =  [[1.7093 -0.0457 167.3234]
 [0.0000 1.4820 77.5282]
 [0.0000 0.0000 1.0000]]

T* =  [0.0000 0.0007 1.0000]

R* =  [[0.9773 -0.2119 0.0000]
 [0.2119 0.9773 0.0007]
 [-0.0001 -0.0006 1.0000]]

M =  [[1.6373 -0.5148 167.3233 167.3233]
 [0.3031 1.3983 77.5292 77.5292]
 [-0.0001 -0.0006 1.0000 1.0000]]

Mean Square Error =  0.5734


## Implementing RANSAC

In [8]:
def GetDistance(M, point_3D, point_2D):
    d = []
    m1, m2, m3 = M[0][:4], M[1][:4], M[2][:4]    
    for i, j in zip(point_3D, point_2D):
        #convert 3D to 3DH
        pi = np.concatenate([np.array(i), [1]])
        #compute xi & yi using M
        computed_xi = (m1.T.dot(pi)) / (m3.T.dot(pi))
        computed_yi = (m2.T.dot(pi)) / (m3.T.dot(pi))
        d.append(np.sqrt((j[0] - computed_xi) ** 2 + (j[1] - computed_yi) ** 2))
    return d


In [9]:
def ApplyRANSAC(point_3D, point_2D):
    w = 0.5    
    count = 0
    num_inliner = 6
    np.random.seed(0)
    prob, kmax, nmin, nmax = 0, 0, 0, 0
    with open('RANSAC.config', 'r') as conf:
        #probability that at least one of the draws is free from outlier
        prob = float(conf.readline().split()[0])
        #maximum number of draws that can be performed
        kmax = int(conf.readline().split()[0])
        #minimum number of points needed to fit model
        nmin = int(conf.readline().split()[0])
        #maximun number of points needed to fit model
        nmax = int(conf.readline().split()[0])     
    
    A = GetMatrixA(point_3D, point_2D)
    M = GetProjectionMatrixM(A) 
    
    d = GetDistance(M, point_3D, point_2D)
    t = 1.5 * np.median(d)
    
    while(count < kmax):        
        i = np.random.choice(len(point_3D), nmax)
        random_3D_points, random_2D_points = np.array(point_3D)[i], np.array(point_2D)[i]
        
        A = GetMatrixA(random_3D_points, random_2D_points)
        M = GetProjectionMatrixM(A)     
        
        d = GetDistance(M, point_3D, point_2D)
        inliner_points = []
        for i, d in enumerate(d):
            if d < t:
                inliner_points.append(i)
        
        if len(inliner_points) >= num_inliner:
            num_inliner = len(inliner_points)
            inliner_3D_points, inliner_2D_points = np.array(point_3D)[inliner_points], np.array(point_2D)[inliner_points]            
            A = GetMatrixA(inliner_3D_points, inliner_2D_points)
            M = GetProjectionMatrixM(A)  
            d = GetDistance(M, point_3D, point_2D)
            
        w = float(len(inliner_points))/float(len(point_2D))
        kmax = float(math.log(1 - prob)) / np.absolute(math.log(1 - (w ** nmax)))
        t = 1.5 * np.median(d)
        count += 1;
        
    print("\nNumber of inliers", num_inliner)
    return A, M


In [10]:
#Camera Calibration with RANSAC
Non_Planar_Calibration('points_3Dto2D.txt', True)




*************************************************************

Total Point Pairs =  49

Camera Parameters from Non-Planar Camera Calibration with RANSAC: 

Number of inliers 28

M^ =  [[0.0062 -0.0020 0.6413 0.6413]
 [0.0011 0.0053 0.2978 0.2978]
 [-0.0000 -0.0000 0.0038 0.0038]]

u0 =  167.4372 		v0 =  77.7658

alpha u =  1.704 	alpha v =  1.4773

s =  -0.0482 		ρ =  261.099

K* =  [[1.7040 -0.0482 167.4372]
 [0.0000 1.4773 77.7658]
 [0.0000 0.0000 1.0000]]

T* =  [0.0000 0.0007 1.0000]

R* =  [[0.9776 -0.2106 0.0000]
 [0.2106 0.9776 0.0007]
 [-0.0002 -0.0007 1.0000]]

M =  [[1.6295 -0.5155 167.4371 167.4371]
 [0.2990 1.3933 77.7668 77.7668]
 [-0.0002 -0.0007 1.0000 1.0000]]

Mean Square Error =  0.594


## Testing With Provided Files

In [11]:
#input point pairs
def ReadTestPoints(f1,f2):    
    point_3D, point_2D = [], []
    with open(f1) as f:
        point_pairs = f.readlines()
        for i in point_pairs:
            pt = i.split()
            point_3D.append([float(p) for p in pt[:3]])
    with open(f2) as f:
        point_pairs = f.readlines()
        for i in point_pairs:
            pt = i.split()
            point_2D.append([float(p) for p in pt[:2]])
    return point_3D, point_2D

def CreateTestFile(filename, point_3D, point_2D):
    f = open(filename,"w+")
    for i in range(len(point_3D)):
        X = str(point_3D[i][0])
        Y = str(point_3D[i][1])
        Z = str(point_3D[i][2])
        x = str(point_2D[i][0])
        y = str(point_2D[i][1])                
        line = X + " " + Y + " " + Z + " " + x + " " + y
        if (i != 0):
            line = "\n" + line
        f.write(line)
    f.close()    
    

In [12]:
#for test
point_3D, point_2D = ReadTestPoints('ncc-worldPt.txt','ncc-imagePt.txt')
CreateTestFile('test1_3Dto2D.txt', point_3D, point_2D)
point_3D, point_2D = ReadTestPoints('ncc-worldPt.txt','ncc-noise-0-imagePt.txt')
CreateTestFile('test2_noise_3Dto2D.txt', point_3D, point_2D)
point_3D, point_2D = ReadTestPoints('ncc-worldPt.txt','ncc-noise-1-imagePt.txt')
CreateTestFile('test3_noise_3Dto2D.txt', point_3D, point_2D)
            

In [13]:
#test 1
Non_Planar_Calibration('test1_3Dto2D.txt', False)
#with RANSAC
Non_Planar_Calibration('test1_3Dto2D.txt', True)




*************************************************************

Total Point Pairs =  268

Camera Parameters from Non-Planar Camera Calibration: 

M^ =  [[0.0016 -0.0006 0.0005 -0.8000]
 [-0.0004 -0.0005 0.0015 -0.6000]
 [0.0000 0.0000 0.0000 -0.0025]]

u0 =  320.0002 		v0 =  240.0

alpha u =  652.1741 	alpha v =  652.1741

s =  -0.0 		ρ =  -419526.1371

K* =  [[652.1741 -0.0000 320.0002]
 [0.0000 652.1741 240.0000]
 [0.0000 0.0000 1.0000]]

T* =  [-0.0003 0.0000 1048.8090]

R* =  [[-0.7682 0.6402 0.0000]
 [0.4273 0.5127 -0.7447]
 [-0.4767 -0.5721 -0.6674]]

M =  [[-653.5681 234.4468 -213.5756 335618.9055]
 [164.2417 197.0901 -645.8414 251714.1620]
 [-0.4767 -0.5721 -0.6674 1048.8090]]

Mean Square Error =  0.0


*************************************************************

Total Point Pairs =  268

Camera Parameters from Non-Planar Camera Calibration with RANSAC: 

Number of inliers 193

M^ =  [[0.0016 -0.0006 0.0005 -0.8000]
 [-0.0004 -0.0005 0.0015 -0.6000]
 [0.0000 0.0000 0.0000 -

In [14]:
#test 2 with noise
Non_Planar_Calibration('test2_noise_3Dto2D.txt', False)
#with RANSAC
Non_Planar_Calibration('test2_noise_3Dto2D.txt', True)




*************************************************************

Total Point Pairs =  268

Camera Parameters from Non-Planar Camera Calibration: 

M^ =  [[0.0016 -0.0005 0.0005 -0.8009]
 [-0.0004 -0.0005 0.0015 -0.5987]
 [0.0000 0.0000 0.0000 -0.0025]]

u0 =  315.7238 		v0 =  226.5402

alpha u =  633.7365 	alpha v =  634.9075

s =  -0.5544 		ρ =  -409477.3567

K* =  [[633.7365 -0.5544 315.7238]
 [0.0000 634.9075 226.5402]
 [0.0000 0.0000 1.0000]]

T* =  [6.9586 20.4681 1024.8522]

R* =  [[-0.7711 0.6367 -0.0054]
 [0.4202 0.5025 -0.7556]
 [-0.4783 -0.5849 -0.6551]]

M =  [[-639.9292 218.5470 -209.8369 327968.8058]
 [158.4515 186.5478 -628.1091 245165.5807]
 [-0.4783 -0.5849 -0.6551 1024.8522]]

Mean Square Error =  4.6167


*************************************************************

Total Point Pairs =  268

Camera Parameters from Non-Planar Camera Calibration with RANSAC: 

Number of inliers 173

M^ =  [[-0.0016 0.0006 -0.0005 0.8015]
 [0.0004 0.0005 -0.0015 0.5980]
 [-0.0000 -0.000

In [15]:
#test 3 with noise
Non_Planar_Calibration('test3_noise_3Dto2D.txt', False)
#with RANSAC
Non_Planar_Calibration('test3_noise_3Dto2D.txt', True)




*************************************************************

Total Point Pairs =  268

Camera Parameters from Non-Planar Camera Calibration: 

M^ =  [[-0.0016 0.0004 -0.0006 0.8015]
 [0.0003 0.0003 -0.0015 0.5980]
 [-0.0000 -0.0000 -0.0000 0.0025]]

u0 =  312.0156 		v0 =  212.2823

alpha u =  491.3009 	alpha v =  495.9305

s =  -5.5191 		ρ =  337380.173

K* =  [[491.3009 -5.5191 312.0156]
 [0.0000 495.9305 212.2823]
 [0.0000 0.0000 1.0000]]

T* =  [13.7633 44.8308 845.7324]

R* =  [[-0.7721 0.6352 -0.0161]
 [0.4134 0.4829 -0.7719]
 [-0.4826 -0.6027 -0.6355]]

M =  [[-532.2213 121.3824 -201.9098 270396.2092]
 [102.5492 111.5615 -517.7340 201767.0096]
 [-0.4826 -0.6027 -0.6355 845.7324]]

Mean Square Error =  52.8994


*************************************************************

Total Point Pairs =  268

Camera Parameters from Non-Planar Camera Calibration with RANSAC: 

Number of inliers 8

M^ =  [[-0.0069 -0.0036 -0.0008 0.6000]
 [-0.0040 -0.0014 0.0034 -0.8000]
 [-0.0000 -0.0000

C:\Users\imcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in double_scalars
